In [5]:
import sqlite3
from datetime import datetime


def init_db():
    conn = sqlite3.connect("tweets.db")
    cursor = conn.cursor()
    cursor.execute(open("schema.sql", "r").read())
    conn.commit()
    conn.close()


def insert_tweet(tweet_id, text, sentiment):
    conn = sqlite3.connect("tweets.db")
    cursor = conn.cursor()
    cursor.execute(
        "INSERT OR IGNORE INTO tweets (tweet_id, text, sentiment, created_at) VALUES (?, ?, ?, ?)",
        (tweet_id, text, sentiment, datetime.now()),
    )
    conn.commit()
    conn.close()

In [8]:
import tweepy
from textblob import TextBlob
import sqlite3
from datetime import datetime
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import os

# Twitter API setup
consumer_key = "jTWP9DLyqYarBCsnGCPeVrKED"
consumer_secret = "AAG89gyMR2eSS3ihH8jJITxbrUvaA5NQIjd47r1KaPQ0MH2wdg"
access_token = "1791885271752945664-rYNNZElfoGLuWTG7lJ1u80ql7rH6tL"
access_token_secret = "mlLNCyxsSt7AxnUQQ9aNN0cfVA2etf9c5XOiIcXrfaKho"

auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret
)
api = tweepy.API(auth)


# Database setup
def init_db():
    conn = sqlite3.connect("tweets.db")
    cursor = conn.cursor()
    cursor.execute(open("schema.sql", "r").read())
    conn.commit()
    conn.close()


def insert_tweet(tweet_id, text, sentiment):
    conn = sqlite3.connect("tweets.db")
    cursor = conn.cursor()
    cursor.execute(
        "INSERT OR IGNORE INTO tweets (tweet_id, text, sentiment, created_at) VALUES (?, ?, ?, ?)",
        (tweet_id, text, sentiment, datetime.now()),
    )
    conn.commit()
    conn.close()


# Sentiment analysis
def analyze_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return "positive"
    elif analysis.sentiment.polarity == 0:
        return "neutral"
    else:
        return "negative"


# Fetch and process tweets
def fetch_and_store_tweets(keyword, count=100):
    tweets = api.search_tweets(q=keyword, lang="en", count=count)
    for tweet in tweets:
        sentiment = analyze_sentiment(tweet.text)
        insert_tweet(tweet.id_str, tweet.text, sentiment)


# Initialize database
init_db()

# Dash app setup
app = dash.Dash(__name__)


def fetch_data():
    conn = sqlite3.connect("tweets.db")
    df = pd.read_sql_query("SELECT * FROM tweets", conn)
    conn.close()
    return df


app.layout = html.Div(
    children=[
        html.H1(children="Twitter Sentiment Analysis"),
        dcc.Graph(id="sentiment-graph"),
        dcc.Interval(
            id="interval-component",
            interval=60 * 1000,  # in milliseconds
            n_intervals=0,
        ),
    ]
)


@app.callback(
    dash.dependencies.Output("sentiment-graph", "figure"),
    [dash.dependencies.Input("interval-component", "n_intervals")],
)
def update_graph(n):
    fetch_and_store_tweets("Python", 100)  # Fetch new tweets and update DB
    df = fetch_data()
    fig = px.histogram(df, x="sentiment", title="Sentiment Analysis of Tweets")
    return fig


if __name__ == "__main__":
    app.run_server(debug=True)

FileNotFoundError: [Errno 2] No such file or directory: 'schema.sql'

In [9]:
import tweepy
from textblob import TextBlob
import sqlite3
from datetime import datetime
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import os

# Twitter API setup
consumer_key = "jTWP9DLyqYarBCsnGCPeVrKED"
consumer_secret = "AAG89gyMR2eSS3ihH8jJITxbrUvaA5NQIjd47r1KaPQ0MH2wdg"
access_token = "1791885271752945664-rYNNZElfoGLuWTG7lJ1u80ql7rH6tL"
access_token_secret = "mlLNCyxsSt7AxnUQQ9aNN0cfVA2etf9c5XOiIcXrfaKho"

auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret
)
api = tweepy.API(auth)


# Database setup
def init_db():
    conn = sqlite3.connect("tweets.db")
    cursor = conn.cursor()
    cursor.execute(
        """
    CREATE TABLE IF NOT EXISTS tweets (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        tweet_id TEXT UNIQUE,
        text TEXT,
        sentiment TEXT,
        created_at TIMESTAMP
    )
    """
    )
    conn.commit()
    conn.close()


def insert_tweet(tweet_id, text, sentiment):
    conn = sqlite3.connect("tweets.db")
    cursor = conn.cursor()
    cursor.execute(
        "INSERT OR IGNORE INTO tweets (tweet_id, text, sentiment, created_at) VALUES (?, ?, ?, ?)",
        (tweet_id, text, sentiment, datetime.now()),
    )
    conn.commit()
    conn.close()


# Sentiment analysis
def analyze_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return "positive"
    elif analysis.sentiment.polarity == 0:
        return "neutral"
    else:
        return "negative"


# Fetch and process tweets
def fetch_and_store_tweets(keyword, count=100):
    tweets = api.search_tweets(q=keyword, lang="en", count=count)
    for tweet in tweets:
        sentiment = analyze_sentiment(tweet.text)
        insert_tweet(tweet.id_str, tweet.text, sentiment)


# Initialize database
init_db()

# Dash app setup
app = dash.Dash(__name__)


def fetch_data():
    conn = sqlite3.connect("tweets.db")
    df = pd.read_sql_query("SELECT * FROM tweets", conn)
    conn.close()
    return df


app.layout = html.Div(
    children=[
        html.H1(children="Twitter Sentiment Analysis"),
        dcc.Graph(id="sentiment-graph"),
        dcc.Interval(
            id="interval-component",
            interval=60 * 1000,  # in milliseconds
            n_intervals=0,
        ),
    ]
)


@app.callback(
    dash.dependencies.Output("sentiment-graph", "figure"),
    [dash.dependencies.Input("interval-component", "n_intervals")],
)
def update_graph(n):
    fetch_and_store_tweets("Python", 100)  # Fetch new tweets and update DB
    df = fetch_data()
    fig = px.histogram(df, x="sentiment", title="Sentiment Analysis of Tweets")
    return fig


if __name__ == "__main__":
    app.run_server(debug=True)

---------------------------------------------------------------------------
Forbidden                                 Traceback (most recent call last)
Cell In[9], line 104, in update_graph(n=0)
     99 @app.callback(
    100     dash.dependencies.Output("sentiment-graph", "figure"),
    101     [dash.dependencies.Input("interval-component", "n_intervals")],
    102 )
    103 def update_graph(n):
--> 104     fetch_and_store_tweets("Python", 100)  # Fetch new tweets and update DB
    105     df = fetch_data()
    106     fig = px.histogram(df, x="sentiment", title="Sentiment Analysis of Tweets")

Cell In[9], line 66, in fetch_and_store_tweets(keyword='Python', count=100)
     65 def fetch_and_store_tweets(keyword, count=100):
---> 66     tweets = api.search_tweets(q=keyword, lang="en", count=count)
        api = <tweepy.api.API object at 0x000001C806F863F0>
        keyword = 'Python'
        count = 100
     67     for tweet in tweets:
     68         sentiment = analyze_sentiment(tweet